In [1]:
import configparser, os
config = configparser.ConfigParser()
config.read('./keys.ini')
openai_api_key = config['OPENAI']['OPENAI_API_KEY']

#replace the open ai key and googld search api key with your own
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPAPI_API_KEY'] = config['SERPAPI']['SERPAPI_API_KEY']

Model def using tools in the langchain agents for png metadata read, write, and google search of character info

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType, AgentExecutor
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain import SerpAPIWrapper
from PIL import Image
import base64
import json

#endable chat memory for agent to remember info from previous
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.clear();
#NOTE: chatgpt is horrible in using memory, even wrose than open source models. It's caused by its silly attempt to stop people using memory to ask for its prompt.
#never trust chatgpt with memory state. try to finish all the operations in one chain / one agent run.

#read chara info from PNG
def png_read(path:str):
    #path is the path of the original png
    #field is the json field to read from the png
    from PIL import Image
    import base64
    import json
    image = Image.open(path)
    info = image.info
    decoded_info = base64.b64decode(info['chara'])
    parsed_info = json.loads(decoded_info)
    image.close()
    return parsed_info['data']

#write chara info to PNG
def png_write(path: str, field: str, new_data: str, new_path: str):
    #path is the path of the original png
    #field is the json field to write in the new png
    #new_data is the data to write in the new png
    #new_path is the path of the new png
    from PIL import Image
    import base64
    import json
    from PIL.PngImagePlugin import PngInfo
    
    #read the json info from the original png
    image = Image.open(path)
    info = image.info
    decoded_info = base64.b64decode(info['chara'])
    parsed_info = json.loads(decoded_info)
    
    #update the json info from the png
    parsed_info['data'][field] = new_data
    new_json_data = json.dumps(parsed_info)
    new_png_data = base64.b64encode(new_json_data.encode())
    new_chara = new_png_data.decode()
    
    #write the new png with the updated metadata
    metadata = PngInfo()
    metadata.add_text("chara", new_chara)
    
    #if new_path file already exist, add meta data to it, otherwise create a new png
    if os.path.exists(new_path):
        image.close();
        image = Image.open(new_path)
    img = image.resize((400,600))
    img.save(new_path,format="PNG",pnginfo=metadata)
    img.close(); image.close();
    return("PNG saved at: "+new_path)

#google image search, fetch
def image_search(search_term: str):
    #search_term is the term used in google search
    from serpapi import GoogleSearch
    import wget
    import random
    SERPAPI_API_KEY = os.environ.get('SERPAPI_API_KEY')
    search = GoogleSearch({"q": search_term, "tbm": "isch", "api_key": SERPAPI_API_KEY})
    result_list = search.get_dict()['images_results']
    random.shuffle(result_list)
    for image_result in result_list:
        #check the aspect ratio of the image, for a card png it should be around 400 pixel width x 600 height
        width = image_result["original_width"]
        height = image_result["original_height"]
        ratio = width / height
        if (ratio < 0.4) | (ratio > 0.8) | (width < 400) | (height < 600):
            continue;
        #download image     
        link = image_result["original"]
        try:
            #print("link: " + link)
            path = './cache/'+search_term+'.png'
            #if not(os.path.exists(path)):
            #    os.mkdir(path)
            wget.download(link, path)
            break
        except:
            pass
        
    


In [32]:
#langchain agent tool defination
SERPAPI_API_KEY = os.environ.get('SERPAPI_API_KEY')
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
tools = load_tools(["python_repl"])
tools += [
    StructuredTool.from_function(
        func=png_read,
        name="PNG read",
        description="useful for when you need to read the json info from PNG for character cards"
    ),
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to search for answers to a question"
    ),
    StructuredTool.from_function(
        name = "PNG write",
        func = png_write,
        description="useful for when you need to update the json info from the original png and save it to the new png"
    )
]

llm = ChatOpenAI(temperature=0.8, model_name = 'gpt-4')

#use STRUCTURED_CHAT for multiple input tools
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True, 
    memory=memory, 
    agent_kwargs = {
        "memory_prompts": [chat_history],
        "input_variables": ["input", "agent_scratchpad", "chat_history"]
    }
)


In [6]:
#main task: generate a character card PNG
char_name = 'amiya'
#use google search to fetch a character picture and resize it
image_search(char_name)
#copy the template 
png_write('template.png','name',char_name,'cache/'+char_name+'.png')

'PNG saved at: cache/amiya.png'

In [28]:
#main task: generate metadata for a character card PNG using google search
field_list = ['description', 'first_mes', 'mes_example', 'personality', 'scenario']
field_des = {'description': char_name + ' character description', 'first_mes': 'what happended in '+char_name+' first time meeting with you',
'mes_example': 'what are example conversation between '+char_name+' and you',
'name': 'name of the character','personality':'whats the age, gender, body typle, and personality of '+char_name,
'scenario':'whats a scenario when you start to talk with the character '+char_name}
search_domain = 'anime'
for f in field_list:
    memory.clear()
    try:
        agent.run("1. Search in "+search_domain+", "+ field_des[f]+". 2. read the "+f+" field from cache/"+char_name+".png. use the field as an example to rewrite "+f+" field with the info from search, using imagnation if needed. 3. save it into the same file.")
    except:
        pass



> Entering new  chain...
Thought: To complete this task, I will first search for the character description of Amiya in anime. Then, I will read the description field from the existing PNG file of Amiya's character card. After that, I will update the description field with the information from the search and write a new long description. Finally, I will save the updated description in the same PNG file.

Action (1 - Search for Amiya's character description in anime):
```
{
  "action": "Search",
  "action_input": "Amiya character description in anime"
}
```


Observation: Character Info Although Amiya appears to be a young and inexperienced girl, she is universally trusted and highly qualified for her position. Amiya now leads the island in fighting for the future of the Infected and striving to exorcise the dark shadow of Originium from the land.
Thought:The search result provides the character description for Amiya in the anime. Now I can proceed to the next step, which is to read th

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


Question 1: What happened in Amiya's first time meeting with you in anime?

Thought: I don't have access to specific information about events in an anime. I'll need to use the Search tool to find the answer.

Action:
```
{
  "action": "Search",
  "action_input": "Amiya first time meeting with protagonist anime"
}
```


Observation: Ace then met Amiya and later on worked for Rhodes Island. Some time passed and then news about Reunion attacking Chernobog went out.
Thought:Amiya first met the protagonist in the anime when they encountered each other and subsequently worked together at Rhodes Island. After some time, news broke out that Reunion was attacking Chernobog.

Question 2: Sure, I can read the `first_mes` field from the `cache/amiya.png` file. Let me do that.

Thought: I'll use the `PNG read` tool to read the `first_mes` field from the `cache/amiya.png` file.

Action:
```
{
  "action": "PNG read",
  "action_input": {
    "path": "cache/amiya.png"
  }
}
```
Observation: {'alternate

In [40]:
#verify the output png and its metadata
png_read('cache/'+char_name+'.png')

{'alternate_greetings': '1687824515939',
 'avatar': 'none',
 'character_version': 'main',
 'creator': 'auto character card by twilledwave, https://github.com/TwilledWave/AutoCharCard',
 'creator_notes': 'Auto character cards for AI chats, generate the character png files used in Text Generation Webui, SillyTavern, or characterhub',
 'description': 'Although Amiya appears to be a young and inexperienced girl, she is universally trusted and highly qualified for her position. Amiya now leads the island in fighting for the future of the Infected and striving to exorcise the dark shadow of Originium from the land.',
 'extensions': {'chub': {'expressions': None,
   'full_path': 'crustcrunch/Kitty',
   'id': 5230,
   'related_lorebooks': []}},
 'first_mes': '<updated description>',
 'mes_example': "I still can't believe that someone made the offer of buying that one. Also watching a healthy amount of AK fans makes me so happy.",
 'name': 'amiya',
 'personality': 'Young, determined, and courag